In [1]:
%pip install sdv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd

orig_data = pd.read_csv("datafusion/preprocessed_with_id_train.csv")

In [20]:
client_ids = orig_data["user_id"].unique()
gen = np.random.default_rng(0)
train_ids = pd.Series(
    gen.choice(client_ids, size=int(len(client_ids) * args.sample_size), replace=False),
    name="user_id",
)
df_train = df.merge(train_ids, on="user_id")


array([     3,      9,     13, ..., 562312, 562721, 562740])

In [12]:
dataset = orig_data[:200000]

In [13]:
from sdv.metadata import SingleTableMetadata
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(dataset)
metadata.update_columns_metadata({
    "user_id": {"sdtype": "id"},
    "mcc_code": {"sdtype": "categorical"},
})
metadata.set_sequence_key("user_id")
metadata.set_sequence_index("days_since_first_tx")

metadata.validate()

In [14]:
from sdv.sequential import PARSynthesizer
synthesizer = PARSynthesizer(
    metadata,
    context_columns=['customer_age', 'dummy_binclass'],
    epochs=128,
    verbose=True)

synthesizer.fit(dataset)

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:92: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Loss (0.022):  37%|███▋      | 47/128 [12:12<21:02, 15.59s/it]


KeyboardInterrupt: 

In [15]:
synthetic_data = synthesizer.sample(num_sequences=10)
synthetic_data

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:01<00:00,  5.95it/s]


,user_id,days_since_first_tx,mcc_code,currency_rk,transaction_amt,customer_age,time_diff_days,dummy_binclass
0,549629312,8.072075,1,1,9654.712440,2,1.238175,1
1,549629312,9.587831,16,1,-1309.703117,2,1.238175,1
2,549629312,11.587006,16,1,-4202.862647,2,2.648899,1
3,549629312,13.685511,16,1,-2168.348228,2,0.761095,1
4,549629312,14.921109,16,1,-1224.891833,2,1.238175,1
...,...,...,...,...,...,...,...,...
162,221282349,8.761401,16,1,837.654395,2,0.000000,1
163,221282349,10.980607,13,1,-1309.703117,2,1.238175,1
164,221282349,12.216205,11,1,-5262.774564,2,0.321980,1
165,221282349,14.123293,3,1,-613.138521,2,2.685225,1
